# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?


In [3]:
import re
import pandas as pd

In [2]:
# egy sor tisztitasa
def sorTisztitas(sor):
  # () ezen kivul amit kap; van egy (; barmi johet; jon egy ); () ezen kivul amit kap, lehet nulla is 
  # allamnev (egyetem) 
  talalat = re.match('([^\(\)]+)\((.+)\)[^\(\)]*', sor)
  if talalat:
    varos = talalat.groups()[0]
    egyetemek = talalat.groups()[1]

  elif '(' in sor:
    varos, egyetemek = sor.split('(')
  elif ',' in sor:
    varos, egyetemek = sor.split(',')
  else:
    return '', []
  szam = re.search('[0-9]+', varos)
  if szam:
    return '', []
  egyetemek = [ i.strip(' ()') for i in egyetemek.split(',')]

  return varos.strip(), egyetemek

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
def beolvas():
  adat = {'allam':[], 'varos':[], 'egyetem':[]}
  with open('/content/drive/MyDrive/egyetemek.txt', 'r') as txt:
    for sor in txt:
      sor = sor.strip()
      if '[edit]' in sor:
        allam = sor = sor.replace('[edit]', '')
      else:
        sor = re.sub('\[[0-9]+\]', '', sor).strip()

        varos, egyetemek = sorTisztitas(sor)
        for egyetem in egyetemek:
          adat['allam'].append(allam)
          adat['varos'].append(varos)
          adat['egyetem'].append(egyetem)

    df = pd.DataFrame(adat)
    df = df.drop_duplicates()
    #print(df['varos'].head(50))
    #print(df.head(20))
    
    return df

In [13]:
df = beolvas()

legtobb_egyetem = df.groupby('allam').size().sort_values().tail(1)
print(legtobb_egyetem)

legkevesebb_egyetem = df.groupby('allam').size().sort_values().head(1)
print(legkevesebb_egyetem)

allam
New York    64
dtype: int64
allam
Wyoming    1
dtype: int64


2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [15]:
allamok = pd.read_csv('/content/drive/MyDrive/allamok.csv', sep=';')
allamok.head()

,"State,Abbreviation"
0,"Alabama,AL"
1,"Alaska,AK"
2,"Arizona,AZ"
3,"Arkansas,AR"
4,"California,CA"


3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

In [16]:
fo = df.drop_duplicates(['varos']).groupby('allam', as_index=False).size()
fo['fo/egyetemi varos'] = allamok['nepesseg'] / fo['size']
fo.head()

KeyError: ignored

4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [17]:
km = df.drop_duplicates(['varos']).groupby('allam', as_index=False).size()
km['km/egyetemi varos'] = allamok['terulet'] / fo['size']
km.head()

KeyError: ignored

5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

In [14]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
 
df.head()

import plotly.graph_objects as go
 
fig = go.Figure(data=go.Choropleth(
    locations=df['code'], # Spatial coordinates
    z = df['total exports'].astype(float), 
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Millions USD of exports",
))
 
fig.update_layout(
    title_text = '2011 US Agriculture Exports',
    geo_scope='usa', 
)
 
fig.show()

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?

In [ ]:
import plotly.graph_objects as go

t = df.groupby('allam', as_index=False).size()
t['rovidites'] = allamok['rovidites']
 
fig = go.Figure(data=go.Choropleth(
    locations = t['rovidites'],
    z = t['size'], 
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Egyetemek száma az adott államban",
))
 
fig.update_layout(
    title_text = '2011 US Agriculture Exports',
    geo_scope='usa', 
)
 
fig.show()

In [ ]:
#lakossag szerint
import plotly.graph_objects as go

t = df.groupby('allam', as_index=False).size()
t['rovidites'] = allamok['rovidites']
t['fo/egyetemi varos'] = allamok['nepesseg'] / t['size']
 
fig = go.Figure(data=go.Choropleth(
    locations = t['rovidites'],
    z = t['fo/egyetemi varos'], 
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Egyetemek száma az adott államban lakossag szerint",
))
 
fig.update_layout(
    title_text = '2011 US Agriculture Exports',
    geo_scope='usa', 
)
 
fig.show()

In [ ]:
#terulet szerint
import plotly.graph_objects as go

t = df.groupby('allam', as_index=False).size()
t['rovidites'] = allamok['rovidites']
t['km/egyetemi varos'] = allamok['terulet'] / t['size']
 
fig = go.Figure(data=go.Choropleth(
    locations = t['rovidites'],
    z = t['km/egyetemi varos'], 
    locationmode = 'USA-states',
    colorscale = 'Blues',
    colorbar_title = "Egyetemek száma az adott államban lakossag szerint",
))
 
fig.update_layout(
    title_text = '2011 US Agriculture Exports',
    geo_scope='usa', 
)
 
fig.show()